# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
from dialoghelper import *
from dialoghelper.inspecttools import *
from dialoghelper.tmux import *

Tools available from dialoghelper: &`[curr_dialog, msg_idx, add_html, find_msg_id, find_msgs, read_msg, del_msg, add_msg, update_msg, msg_insert_line, msg_str_replace, msg_strs_replace, msg_replace_lines]`

Tools available from fastcore.tools: &`[rg, sed, view, create, insert, str_replace, strs_replace, replace_lines]`

In [ ]:
from fastcore.tools import *

# skillhelper Design Specification

## Goal

Build `skillhelper` — a Python package that enables solveit to discover, load, and use Agent Skills (Anthropic's open standard for giving agents capabilities). Skills should also work with Claude Code via filesystem compatibility.

## Agent Skills Standard

Skills are directories with a spec-compliant structure:
```
skill-name/
├── SKILL.md          # Required: YAML frontmatter + markdown instructions
├── scripts/          # Optional: executable code
├── references/       # Optional: additional documentation  
└── assets/           # Optional: templates, resources
```

SKILL.md frontmatter (required fields):
- `name`: kebab-case, 1-64 chars, must match directory name
- `description`: what it does and when to use it, max 1024 chars

Optional fields: `license`, `compatibility`, `metadata`, `allowed-tools`

**Progressive disclosure**: Agents load only metadata at startup, full instructions on activation, resources on demand.

## skillhelper Architecture

### Discovery (multiple sources)
1. **Entry points** — pip-installed skill packages register via `[project.entry-points."skillhelper.skills"]`
2. **Filesystem** — `~/.skills/` (user), `./skills/` (project-local)
3. **Environment** — `SOLVEIT_SKILLS_PATH` for custom paths

### Core Abstractions

```python
@dataclass
class Skill:
    name: str
    description: str
    path: Path
    instructions: str  # Full SKILL.md body
    compatibility: Optional[str]
    allowed_tools: Optional[str]
    metadata: Optional[dict]
    
    def load_reference(self, name: str) -> str: ...
    def run_script(self, name: str, *args) -> str: ...
    def asset_path(self, name: str) -> Path: ...
```

### Tools for solveit AI (exposed via `&` syntax)
- `discover_skills(paths)` — scan and register skills
- `list_skills()` — show available skills with descriptions
- `activate_skill(name)` — load full instructions into context
- `skill_reference(skill_name, ref_name)` — load reference docs
- `skill_asset_path(skill_name, asset_name)` — get asset path
- `skill_tool_info()` — returns `<available_skills>` XML for prompt injection

### Claude Code Compatibility
- `skillhelper sync` command creates symlinks from `~/.skills/` to pip-installed skill locations
- Optional meta-skill teaches Claude Code how to discover skillhelper-managed skills
- Same skill format works in both environments

## Skill Package Structure (for pip-installable skills)

```
my-skill/
├── pyproject.toml
└── src/
    └── my_skill/
        ├── __init__.py      # get_skill_path() → Path to skill/
        └── skill/           # Spec-compliant skill directory
            ├── SKILL.md
            ├── scripts/
            └── references/
```

pyproject.toml entry point:
```toml
[project.entry-points."skillhelper.skills"]
my-skill = "my_skill:get_skill_path"
```

## Development with nbdev

- Skill instructions authored in notebooks, exported to SKILL.md
- Code cells export to scripts/ or Python modules
- Tests verify skill functionality
- Package includes entry points for skillhelper discovery

## Key References

- Agent Skills spec: https://agentskills.io/specification
- Reference implementation: `skills-ref/` in current directory (parser, validator, models)
- dialoghelper patterns: https://answerdotai.github.io/dialoghelper/core.html.md
- Example skills: `../skills/custom_skills/`

## Template Pattern

Follow `dialoghelper` patterns:
- Functions with type annotations + docstrings become tools
- `find_dname()` pattern for context discovery
- `call_endp()` for API communication where needed
- Entry in tool info function for solveit integration

## References
- [Agent Skills Homepage](https://agentskills.io/home)
- [Agent Skill Specification](https://agentskills.io/specification)
- [Whhat are Skills](https://agentskills.io/what-are-skills)
- [Integrate Skills](https://agentskills.io/integrate-skills)

Greetings! I'm Chuck and I want to build skillhelper using the fast.ai/answer.ai/solveit approach of literate and exploratory programming. I'm ok with you writeing the code, but it should be in small chunks and use the https://docs.fast.ai/dev/style.html style guide that you should look up if you are not familiar with how to code in this style. Don't write any code just yet, but think through what we want to do here.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Greetings Chuck! Great to work with you on skillhelper.

Let me first check the fast.ai style guide to ensure I'm aligned with the coding conventions:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_01DNo6Uk4Ug1UuETFGv2bbGK",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://docs.fast.ai/dev/style.html"
    }
  },
  "result": "'[ fastai ](../index.html)\\n\\n  * [ __ ](https://github.com/fastai/fastai)\\n\\n\\n\\n[ __ ]( \"Toggle reader mode\")\\n\\n__\\n\\n  1. [fastai Development](../dev-setup.html)\\n  2. [fastai coding style](../dev/style.html)\\n\\n\\n\\n  * [ Welcome to fastai](../index.html)\\n\\n  * [ Quick start](../quick_start.html)\\n\\n  * Tutorials __\\n\\n    * [ Tutorials](../tutorial.html)\\n\\n    * Beginner __\\n\\n      * [ Computer vision intro](../tutorial.vision.html)\\n\\n      * [ Text transfer learning](../tutorial.text.html)\\n\\n      * [ Tabular training](../tutorial.tabular.html)\\n\\n      * [ Collaborative filtering t<TRUNCATED>"
}
```

</details>

Good, I've reviewed the fast.ai style guide. Now let me think through what we want to accomplish with skillhelper.

**Key design goals based on the spec:**

1. **Progressive disclosure** — Load only metadata at startup, full instructions on activation, resources on demand
2. **Multiple discovery sources** — Entry points, filesystem (`~/.skills/`, `./skills/`), and env vars
3. **Claude Code compatibility** — Skills should work in both solveit and Claude Code

**Proposed development order (small steps):**

1. **Parse SKILL.md** — Start with the core: a function to read and parse YAML frontmatter + markdown body from a single SKILL.md file
2. **Skill dataclass** — Simple dataclass holding parsed data + path
3. **Discover from filesystem** — Scan directories for valid skills
4. **Entry point discovery** — Load skills registered via pip packages
5. **Tool functions** — `list_skills()`, `activate_skill()`, etc. for solveit integration
6. **CLI sync command** — For Claude Code compatibility

**Questions for you before we start coding:**

1. Should we look at the reference implementation in `skills-ref/` first to see how they handle SKILL.md parsing, or start fresh?
2. Do you want to begin with the SKILL.md parser, or would you prefer to start with the `Skill` dataclass structure?

Ok. I have added the agentskills repository as a git submodule. the skills-ref is in the submodule directory.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Let's take a look at what's in the skills-ref directory to understand the reference implementation:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_01TBYXEsVjYigZsB4qZJpSpQ",
  "call": {
    "function": "view",
    "arguments": {
      "path": "skills-ref"
    }
  },
  "result": "'Error: File not found: /app/data/research/skillhelper/nbs/skills-ref'"
}
```

</details>

Let me check the current directory structure to find where the submodule is located:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_01RZ2VZ86rj9wShFwuPTmp6s",
  "call": {
    "function": "view",
    "arguments": {
      "path": "."
    }
  },
  "result": "'Directory contents of /app/data/research/skillhelper/nbs:\\n/app/data/research/skillhelper/nbs/styles.css\\n/app/data/research/skillhelper/nbs/_quarto.yml\\n/app/data/research/skillhelper/nbs/00_core.ipynb\\n/app/data/research/skillhelper/nbs/index.ipynb\\n/app/data/research/skillhelper/nbs/nbdev.yml'"
}
```

</details>

Let me check the parent directory:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_014YdpeDJXAekFZofnP6QdzJ",
  "call": {
    "function": "view",
    "arguments": {
      "path": ".."
    }
  },
  "result": "'Directory contents of /app/<TRUNCATED>"
}
```

</details>

Great! Found it at `../agentskills/skills-ref`. Let me look at the key files in the reference implementation:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_0181aqimKGrW62ij553mrYNd",
  "call": {
    "function": "view",
    "arguments": {
      "path": "../agentskills/skills-ref/src/skills_ref"
    }
  },
  "result": "'Directory contents of /app/<TRUNCATED>"
}
```

</details>

Perfect! The reference implementation has exactly what we need. The key files are:
- `models.py` - Data structures for skills
- `parser.py` - SKILL.md parsing logic
- `validator.py` - Validation rules
- `prompt.py` - For generating prompts (progressive disclosure)

Should I look at these files to understand their approach, or would you prefer to start fresh with our own implementation following fast.ai style?

I'm fine using these as reference, but we should re-implement them using the fast.ai style and fastcore.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let's start by looking at their models to understand the data structures, then we'll reimplement using fast.ai style and fastcore:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_01Eg3WFbN9wGReuQJbs1PnfS",
  "call": {
    "function": "view",
    "arguments": {
      "path": "../agentskills/skills-ref/src/skills_ref/models.py"
    }
  },
  "result": "'\"\"\"Data models for Agent Skills.\"\"\"\\n\\nfrom dataclasses import dataclass, field\\nfrom typing import Optional\\n\\n\\n@dataclass\\nclass SkillProperties:\\n    \"\"\"Properties parsed from a skill\\'s SKILL.md frontmatter.\\n\\n    Attributes:\\n        name: Skill name in kebab-case (required)\\n        description: What the skill does and when the model should use it (required)\\n        license: License for the skill (optional)\\n        compatibility: Compatibility information for the skill (optional)\\n        allowed_tools: Tool patterns the skill requires (optional, experimental)\\n        metadata: Key<TRUNCATED>"
}
```

</details>

Now let's look at the parser to see how they extract YAML frontmatter:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_01J1Y6Wi1mHRSXqk3ZxUEZE6",
  "call": {
    "function": "view",
    "arguments": {
      "path": "../agentskills/skills-ref/src/skills_ref/parser.py"
    }
  },
  "result": "'\"\"\"YAML frontmatter parsing for SKILL.md files.\"\"\"\\n\\nfrom pathlib import Path\\nfrom typing import Optional\\n\\nimport strictyaml\\n\\nfrom .errors import ParseError, ValidationError\\nfrom .models import SkillProperties\\n\\n\\ndef find_skill_md(skill_dir: Path) -> Optional[Path]:\\n    \"\"\"Find the SKILL.md file in a skill directory.\\n\\n    Prefers SKILL.md (uppercase) but accepts skill.md (lowercase).\\n\\n    Args:\\n        skill_dir: Path to the skill directory\\n\\n    Returns:\\n        Path to the SKILL.md file, or None if not found\\n    \"\"\"\\n    for name in (\"SKILL.md\", \"skill.md\"):\\n        path <TRUNCATED>"
}
```

</details>

Good! Now I understand their approach. They use `strictyaml` for parsing. 

**Key observations:**
1. They split models (SkillProperties) from the full Skill concept
2. They use simple string splitting for frontmatter extraction
3. They have separate parsing and validation steps

**For our fast.ai style implementation, should we:**
1. Start with a simple frontmatter parser function (using fastcore utilities where applicable)?
2. Or begin with the `Skill` dataclass that will hold everything we need?

Which would you like to tackle first?

So. We want to test this as we go using the fast.ai approach. We is there an example skill we should create in the repository?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good thinking! Having a test skill will help us develop incrementally. Let me check if there are example skills in the agentskills repo:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_017xABFEix13bBxYRg77cMTQ",
  "call": {
    "function": "rg",
    "arguments": {
      "argstr": "-l SKILL.md ../agentskills"
    }
  },
  "result": "'../a<TRUNCATED>"
}
```

</details>

Let me look for actual example skills in their test fixtures:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_011XrEv3k6sjMGhLtLSuu3BJ",
  "call": {
    "function": "view",
    "arguments": {
      "path": "../agentskills/skills-ref/tests"
    }
  },
  "result": "'Directory contents of /app/data/research/skillhelper/agentskills/skills-ref/tests:\\n/app/data/research/skillhelper/agentskills/skills-ref/tests/test_prompt.py\\n/app/data/research/skillhelper/agentskills/skills-ref/tests/__init__.py\\n/app/data/research/skillhelper/agentskills/skills-ref/tests/test_parser.py\\n/app/data/research/skillhelper/agentskills/skills-ref/tests/test_validator.py'"
}
```

</details>



<details class='tool-usage-details'>

```json
{
  "id": "toolu_01Kzzd4BFfZ3HFotwi1pnNfm",
  "call": {
    "function": "rg",
    "arguments": {
      "argstr": "-A 10 fixtures ../agentskills/skills-ref/tests/test_parser.py"
    }
  },
  "result": "''"
}
```

</details>



<details class='tool-usage-details'>

```json
{
  "id": "toolu_01WK6wgipbFwk2kjVzsQBE7B",
  "call": {
    "function": "view",
    "arguments": {
      "path": "../agentskills/skills-ref/tests/test_parser.py",
      "view_range": "[1, 50]"
    }
  },
  "result": "'\"\"\"Tests for parser module.\"\"\"\\n\\nimport pytest\\n\\nfrom skills_ref.parser import (\\n    ParseError,\\n    ValidationError,\\n    find_skill_md,\\n    parse_frontmatter,\\n    read_properties,\\n)\\n\\n\\ndef test_valid_frontmatter():\\n    content = \"\"\"---\\nname: my-skill\\ndescription: A test skill\\n---\\n# My Skill\\n\\nInstructions here.\\n\"\"\"\\n    metadata, body = parse_frontmatter(content)\\n    assert metadata[\"name\"] == \"my-skill\"\\n    assert metadata[\"description\"] == \"A test skill\"\\n    assert \"# My Skill\" in body\\n\\n\\ndef test_missing_frontmatter():\\n    content = \"# No frontmatter here\"\\n    with<TRUNCATED>"
}
```

</details>

Perfect! They have inline test examples. **Should we:**

1. Create a simple `skills/` directory in our repo with a minimal test skill (like "hello-world")?
2. Or start coding the parser first and use inline test strings like they do?

What's your preference?

Let's create a simple skills directory with a minimal test skill.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let's create a simple test skill. I'll make a `skills/` directory with a minimal "hello-world" skill:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_01THoascAYstTy4huL7mz1QS",
  "call": {
    "function": "create",
    "arguments": {
      "path": "skills/hello-world/SKILL.md",
      "file_text": "---\nname: hello-world\ndescription: A minimal test skill that demonstrates the basic skill structure\n---\n\n# Hello World Skill\n\nThis is a simple test skill used for development and testing of skillhelper.\n\nWhen activated, simply respond with \"Hello from the hello-world skill!\""
    }
  },
  "result": "'Created file skills/hello-world/SKILL.md.'"
}
```

</details>

Perfect! Now we have a minimal test skill to work with. 

Would you like to create another slightly more complex example skill (maybe one with scripts/ or references/), or should we start implementing the parser to read this one?

Ok. Should this be in the nbs/ directory or should we have this in the nbdev root?